In [1]:
import pandas as pd
import numpy as np
from scipy.stats import beta, weibull_min
from datetime import timedelta

C:\Users\carlv\AppData\Local\Temp\ipykernel_25020\2409071396.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [91]:
df_path='../data/data_wind_solar_2024_25.csv'

df = pd.read_csv(df_path, sep=';')
df.drop('endTime', inplace=True, axis=1)
df.rename(columns={'startTime': 'Datum'}, inplace=True)
df["Datum"] = pd.to_datetime(df["Datum"], format='mixed')
df["Datum"] = df['Datum'].dt.tz_localize(None)
df.set_index('Datum', inplace=True)
df.columns = ['Wind', 'Solar']
day = None
if day is None:
    day = df.index[0] + timedelta(days=int(365/2))  # middle of the year
day = pd.to_datetime(day)

# normalize wind and solar to [0,1]
df = df.copy()
df['Wind'] = df['Wind'].clip(lower=0) / (df['Wind'].max() + 1e-9)
df['Solar'] = df['Solar'].clip(lower=0) / (df['Solar'].max() + 1e-9)


In [92]:
# Select ±1 month window
start = day - pd.DateOffset(months=1)
end = day + pd.DateOffset(months=1)
df_window = df.loc[start:end]

In [ ]:
# resample to hourly data
df_s = df_window['Solar']
df_s = df_s.groupby([df_s.index.hour]).apply(list)

In [84]:
def get_alpha_beta_mean(data, thresh=0.02):
    if np.mean(data) < thresh:
        return 0.0
    a, b, loc, scale = beta.fit(data)
    mean = beta.mean(a, b, loc=loc, scale=scale)
    return mean

In [86]:
# df_s.apply(lambda x: get_alpha_beta_mean(x))

In [93]:
df_w = df_window['Wind']
df_w = df_w.groupby([df_w.index.hour]).mean()

In [ ]:
df_bat = pd.read_csv('../data/battery_soc_profile.csv', sep=';')


In [104]:
from time_varying_dc_gain import get_wind_solar_dc_gains

In [106]:
path='../data/data_wind_solar_2024_25.csv'
df = pd.read_csv(path, sep=';')
df.drop('endTime', inplace=True, axis=1)
df.rename(columns={'startTime': 'Datum'}, inplace=True)
df["Datum"] = pd.to_datetime(df["Datum"], format='mixed')
df.set_index('Datum', inplace=True)
df.columns = ['Wind', 'Solar']
# normalize by max values
max_vals = df.max()
df_mean = df / max_vals

In [137]:
df_bat = pd.read_csv('../data/battery_soc_profile.csv', sep=';', index_col=0)
df_bat.loc[24, 'Battery_SOC (MWh)']

3.5

In [120]:
idx=101
print(df_mean.index[idx].weekday(), df_mean.index[idx].hour, df_mean.index[idx].minute)

3 23 15
